In [2]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import time
import torch
from environment import Environment
from maddpg import MADDPGAgentTrainer
import tensorflow as tf
import tensorflow.contrib.layers as layers

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [3]:
# Initializing pybullet
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)




# Creating robots
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

start = (0.25,0.25)

static = []

#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,-0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [1.25,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#target = static[3]

goal = (1.25, 0.75)

# For milestone 2 only - setting all non-player targets to have no collision with player robot
# Doesn't work when dragging into each other but might still work?
#pybullet.setCollisionFilterGroupMask(player, target, 0, 0)

#for robot in static:
#    pybullet.setCollisionFilterGroupMask(player, robot, 0, 0)

# Setting colors to distinguish robots. BLUE for player, GREEN for obstacle, RED for target
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])
#for robot in static:
#    pybullet.changeVisualShape(robot, 0, rgbaColor=[0,1,0,1])
#pybullet.changeVisualShape(target, 0, rgbaColor=[1,0,0,1])



# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
            cameraEyePosition=[0.75,0.25,3],
            cameraTargetPosition=[0.75,0.25,0],
            cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.1,
            farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)

# Getting position and orientation
print(pybullet.getBasePositionAndOrientation(player))
#print(pybullet.getBasePositionAndOrientation(static[0]))
#print(pybullet.getBasePositionAndOrientation(static[1]))
#print(pybullet.getBasePositionAndOrientation(static[2]))
#print(pybullet.getBasePositionAndOrientation(target))


# Start simulation
pybullet.setRealTimeSimulation(1)

((0.25, 0.25, 0.0), (0.0, 0.0, 0.0, 1.0))


In [ ]:
def mlp_model(input, num_outputs, scope, reuse=False, num_units=64, rnn_cell=None):
    with tf.variable_scope(scope, reuse=reuse):
        out = input
        out = layers.fully_connected(out, num_outputs=num_units, activation_fn=tf.nn.relu)
        out = layers.fully_connected(out, num_outputs=num_units, activation_fn=tf.nn.relu)
        out = layers.fully_connected(out, num_outputs=num_outputs, activation_fn=None)
        return out

model = mlp_model

In [ ]:
# Required:
#Max_episode_len

arglist = {'--max-episode-len': 25, '--num-episodes': 60000, '--num-adversaries': 0, '--good-policy': 'maddpg', '--lr': 1e-2, '--gamma': 0.95, '--batch-size': 1024, '--num-units': 64}

In [ ]:
env = Environment()
agents = env.agents
trainer = MADDPGAgentTrainer
trainers = []
for i in range(env.n):
    trainers.append(trainer(agents[i].name, model, env.observation_space, env.action_space, i, arglist, 'False'))

#U.initialize()

episode_rewards = [0.0]
agent_rewards = [[0.0] for _ in range(env.n)]
final_ep_rewards = []
final_ep_ag_rewards = []
agent_info = [[]]
saver = tf.train.Saver()
obs_n = env.reset()
episode_step = 0
train_step = 0
t_start = time.time()

while True:
    action_n = [agent.action(obs) for agent, obs in zip(trainers, obs_n)]
    new_obs_n, rew_n, done_n = env.step(action_n)
    episode_step += 1
    done = all(done_n)
    terminal = (episode_step >= arglist.max_episode_len)
    for i, agent in enumerate(trainers):
        agent.experience(obs_n[i], action_n[i], rew_n[i], new_obs_n[i], done_n[i], terminal)
    obs_n = new_obs_n

    for i, rew in enumerate(rew_n):
        episode_rewards[-1] += rew
        agent_rewards[i][-1] += rew

    if done or terminal:
        obs_n = env.reset()
        episode_step = 0
        episode_rewards.append(0)
        for a in agent_rewards:
            a.append(0)
        agent_info.append([[]])

    train_step += 1

    loss = None
    for agent in trainers:
        agent.preupdate()
    for agent in trainers:
        loss = agent.update(trainers, train_step)

    



In [42]:
# RRT
samplingArea = [0.25, 1.25, -0.25, 0.75]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.7042946502862929, 0.35086882353507776, 2.8336094648785703]
[0. 0. 0.]
[0.7042946502862929, 0.35086882353507776, 2.8336094648785703]
[0. 0. 0.]
[0.7042946502862929, 0.35086882353507776, 2.8336094648785703]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 

[1.180948201606115, 0.7440951048117771, 3.7421803451344644]
[0. 0. 0.]
[1.1301210523447032, -0.005897453695272392, 0.07285074370681775]
[0. 0. 0.]
[0.8244174341030303, 0.26819787435551823, 3.242941930187362]
[0. 0. 0.]
[0.986267293941548, -0.14506241643511697, 1.39231882313486]
[0. 0. 0.]
[1.1124274336794022, -0.13501227587084397, 5.234474365644098]
[0. 0. 0.]
[0.4542404950494233, 0.2973184673154089, 3.736465520073613]
[0. 0. 0.]
[0.389120060467974, 0.7054856314091241, 4.433377634910545]
[0. 0. 0.]
[0.8701489482435615, 0.554739072605888, 3.930032893829044]
[0. 0. 0.]
[1.145671725292048, 0.38111125036822036, 4.825923833336912]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]


[1.180948201606115, 0.7440951048117771, 3.7421803451344644]
[0. 0. 0.]
[1.1301210523447032, -0.005897453695272392, 0.07285074370681775]
[0. 0. 0.]
[0.8244174341030303, 0.26819787435551823, 3.242941930187362]
[0. 0. 0.]
[0.986267293941548, -0.14506241643511697, 1.39231882313486]
[0. 0. 0.]
[1.1124274336794022, -0.13501227587084397, 5.234474365644098]
[0. 0. 0.]
[0.4542404950494233, 0.2973184673154089, 3.736465520073613]
[0. 0. 0.]
[0.389120060467974, 0.7054856314091241, 4.433377634910545]
[0. 0. 0.]
[0.8701489482435615, 0.554739072605888, 3.930032893829044]
[0. 0. 0.]
[1.145671725292048, 0.38111125036822036, 4.825923833336912]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.3249992842751487, 0.4910223617774516, 0.9683621126708308]
[0. 0. 0.]
[0.9989882141231782, -0.043456649804659464, 2.411591907055432]
[0. 0. 0.]
[0.9639141560925881, -0.030490679518250907, 3.255388255858381]
[0. 0. 0.]
[0.9306695050796961, 0.7489756166732197, 1.3526034517703913]
[0.

[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.9306695050796961, 0.7489756166732197, 1.3526034517703913]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.7042946502862929, 0.3508688235350

[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[0.389120060467974, 0.7054856314091241, 4.433377634910545]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.8860108500645538, 0.28387046076882994, 0.7915552736586884]
[0. 0. 0.]
[0.862307851088639, 0.28398930937387623, 3.2030241628312157]
[0. 0. 0.]
[0.7042946502862929, 0.35086882353507776, 2.8336094648785703]
[0. 0. 0.]
[1.25, 0.75, 0]
[0. 0. 0.]
[0.82143

...

[0. 0. 0.]
[0.5891415071227228, 0.02457885504315893, 0.627660573141021]
[0. 0. 0.]
[0.9188990370343916, 0.606399036222519, 2.516230397333817]
[0. 0. 0.]
[1.1400552898805982, -0.09138888880828233, 5.927350937664264]
[0. 0. 0.]
[0.43745033353568785, 0.4762054198139146, 0.6087728200488924]
[0. 0. 0.]
[1.1660214081083362, 0.7237207598411814, 2.4832173408579195]
[0. 0. 0.]
[0.3362752580140286, 0.668575298860726, 1.5017858039746192]
[0. 0. 0.]
[0.9282750369344223, 0.43905032397367283, 5.097033772015708]
[0. 0. 0.]
[0.8941548685139429, 0.5994958326876048, 5.379808371574054]
[0. 0. 0.]
[0.8548697225998936, 0.33076333448120754, 0.45180217300281456]
[0. 0. 0.]
[0.7188085976242762, 0.05071474677301446, 6.135691308055657]
[0. 0. 0.]
[0.7675704083519923, 0.2822814284353189, 1.732080585340406]
[0. 0. 0.]
[0.2640704594897654, 0.5768327288915368, 3.961209522663378]
[0. 0. 0.]
[1.1339649881052498, -0.22382048581368896, 1.0900498877457818]
[0. 0. 0.]
[0.9801579314702596, 0.7083496343870951, 1.1759051823

[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.3249992842751487, 0.4910223617774516, 0.9683621126708308]
[0. 0. 0.]
[0.9989882141231782, -0.043456649804659464, 2.411591907055432]
[0. 0. 0.]
[0.9639141560925881, -0.030490679518250907, 3.255388255858381]
[0. 0. 0.]
[0.9306695050796961, 0.7489756166732197, 1.3526034517703913]
[0. 0. 0.]
[0.770044226072616, 0.3831493038560916, 3.9224941116049474]
[0. 0. 0.]
[1.219760075525859, 0.46961007444932423, 3.175935095718165]
[0. 0. 0.]
[0.4109122985945268, 0.0054313724086344495, 0.973090673036292]
[0. 0. 0.]
[0.9084935431125974, 0.11627752592338347, 5.621671959555256]
[0. 0. 0.]
[0.7243753705710381, 0.6983891952095383, 1.9348874536222467]
[0. 0. 0.]
[0.3775883197276926, 0.5712727595221813, 0.6992793014487951]
[0. 0. 0.]
[1.034545817158462, 0.17953025686845248, 0.28555138076054604]
[0. 0. 0.]
[1.0816552654159524, 0.6630642663063683, 4.689751856282673]
[0. 0. 0.]
[0.458260280625452, 0.7429493570801997, 4.08903213

[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0. 0.]
[0.3686836489412282, 0.6269071030561029, 6.115943433322962]
[0. 0.

[0.986267293941548, -0.14506241643511697, 1.39231882313486]
[0. 0. 0.]
[1.1124274336794022, -0.13501227587084397, 5.234474365644098]
[0. 0. 0.]
[0.4542404950494233, 0.2973184673154089, 3.736465520073613]
[0. 0. 0.]
[0.389120060467974, 0.7054856314091241, 4.433377634910545]
[0. 0. 0.]
[0.8701489482435615, 0.554739072605888, 3.930032893829044]
[0. 0. 0.]
[1.145671725292048, 0.38111125036822036, 4.825923833336912]
[0. 0. 0.]
[0.7910716155040008, 0.3720841150618476, 4.556712769179636]
[0. 0. 0.]
[0.3249992842751487, 0.4910223617774516, 0.9683621126708308]
[0. 0. 0.]
[0.9989882141231782, -0.043456649804659464, 2.411591907055432]
[0. 0. 0.]
[0.9639141560925881, -0.030490679518250907, 3.255388255858381]
[0. 0. 0.]
[0.9306695050796961, 0.7489756166732197, 1.3526034517703913]
[0. 0. 0.]
[0.770044226072616, 0.3831493038560916, 3.9224941116049474]
[0. 0. 0.]
[1.219760075525859, 0.46961007444932423, 3.175935095718165]
[0. 0. 0.]
[0.4109122985945268, 0.0054313724086344495, 0.973090673036292]
[0. 0.

[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866032246095]
[0. 0. 0.]
[0.9345299225714236, -0.09921216240963415, 3.236866

In [43]:
print(path)
path = np.flip(path, 0)
print(path)


[[1.25, 0.75, 0], [1.25, 0.75, 0], [0.25, 0.25, 0]]
[[0.25 0.25 0.  ]
 [1.25 0.75 0.  ]
 [1.25 0.75 0.  ]]


In [81]:
extended_path = []
for i in range(len(path)):
    print(path[i])
for i in range(len(path)):
    if i + 1 != len(path) - 1:
        angle = 0.39
        path_part = list(zip(np.linspace(path[i][0], path[i+1][0], 10), np.linspace(path[i][1], path[i+1][1], 10)))
        for part in path_part:
            extended_path.append((part[0], part[1], angle))
    else:
        break
        
print(extended_path)

[0.25 0.25 0.  ]
[1.25 0.75 0.  ]
[1.25 0.75 0.  ]
[(0.25, 0.25, 0.39), (0.3611111111111111, 0.3055555555555556, 0.39), (0.4722222222222222, 0.3611111111111111, 0.39), (0.5833333333333333, 0.41666666666666663, 0.39), (0.6944444444444444, 0.4722222222222222, 0.39), (0.8055555555555556, 0.5277777777777778, 0.39), (0.9166666666666666, 0.5833333333333333, 0.39), (1.0277777777777777, 0.6388888888888888, 0.39), (1.1388888888888888, 0.6944444444444444, 0.39), (1.25, 0.75, 0.39)]


In [87]:
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [fM * action[0], fM * action[1], 0]
            for x in range(10):
                pybullet.stepSimulation()
                time.sleep(1./240.)
                current_pos = pybullet.getBasePositionAndOrientation(player)[0]
                #pybullet.applyExternalForce(player, -1, action_3d, current_pos, pybullet.WORLD_FRAME)
                pybullet.resetBaseVelocity(player, action_3d)
            pybullet.resetBaseVelocity(player, [0,0,0])

        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [88]:
def getCost(states, target):
    #target = [goal[0], goal[1], 0]
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        costs.append(cost)
    return costs

In [89]:
H = 5
G = 100
T = 50
K = 10
fM = 1

def cem(path):
    pybullet.setRealTimeSimulation(0)
    for i in range(len(path) - 1):
    #for i in range(1000):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        print(state)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1])
            #costs = getCost(new_states, path[1])
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions), key=lambda pair: pair[0])]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            #print(sorted_actions[:K])
            #print(new_actions)
            initial_actions = np.concatenate((sorted_actions[:K], new_actions))
            
        best_action = initial_actions[0][0] # Because no horizon length
        action = [fM * best_action[0], fM * best_action[1], 0]
        #print(action)
        for x in range(10):
            pybullet.stepSimulation()
            time.sleep(1./240.)
            #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
            pybullet.resetBaseVelocity(player, action)
        pybullet.resetBaseVelocity(player, [0,0,0])

            
            


In [90]:
cem(extended_path)
pybullet.setRealTimeSimulation(1)

((0.19716523480499246, 0.24913519955569793, 0.0012710685370691957), (0.0007802876490725206, 0.08106143050642703, -0.009568395133430777, 0.9966628725150782))


KeyboardInterrupt: 

In [85]:
pybullet.resetBasePositionAndOrientation(player, [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

In [215]:
pybullet.disconnect()